<a href="https://colab.research.google.com/github/sujalkhandait/Amazon_clone/blob/main/Twiteer_sentiment_analysis_minor_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


#api to fetch the datasaet

In [ ]:
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.38GB/s]


#unzipping the dataset

In [ ]:
from zipfile import ZipFile
dataset="/content/sentiment140.zip"

with ZipFile(dataset ,"r") as zip:
  zip.extractall()
  print("file is extracted")

file is extracted


#importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

printing stopwords in english

In [ ]:
print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

loading the data from csv file to pandas dataframe

In [ ]:
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")

In [ ]:
#checking the no of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming the colums and reading dataset again
column_names=["target" ,"id" ,"date" , "flag" ,"user" ,"text"]
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", names=column_names ,encoding="ISO-8859-1")

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


convert the targt from 4 to 1


In [ ]:
twitter_data['target'].replace({4: 1}, inplace=True)


/tmp/ipython-input-3815692793.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  twitter_data['target'].replace({4: 1}, inplace=True)


In [ ]:
twitter_data['target'].value_counts()


,count
target,
0,800000
1,800000


stemming :- a process to reduce a word to its root words

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content=re.sub('[^a-zA-Z]', ' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data_sample = twitter_data.head(5000)
twitter_data_sample['stemmed_content'] = twitter_data_sample['text'].apply(stemming)


/tmp/ipython-input-3535894748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data_sample['stemmed_content'] = twitter_data_sample['text'].apply(stemming)


In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].astype(str).apply(stemming)


In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't updat hi facebook by te...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichan i dive mani time for the ball. manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclass no, it' not behav at all. i'm..."


In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


seperates the data and label

In [ ]:
x=twitter_data['stemmed_content'].values
y=twitter_data['target'].values

In [ ]:
print(x)

["@switchfoot http://twitpic.com/2y1zl - awww, that' a bummer. you shoulda got david carr of third day to do it. ;d"
 "is upset that he can't updat hi facebook by text it... and might cri as a result school today also. blah!"
 '@kenichan i dive mani time for the ball. manag to save 50% the rest go out of bound'
 ... 'are you readi for your mojo makeover? ask me for detail'
 'happi 38th birthday to my boo of alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(y)

[0 0 0 ... 1 1 1]


splitting the data to training data and test data

In [ ]:

X_train ,X_test , Y_train ,Y_test =train_test_split(x,y, test_size=0.2, stratify=y ,random_state=2)

In [ ]:
print(x.shape , X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['about to watch saw iv and drink a lil wine' "@hatermagazin i'm in!"
 'even though it my favourit drink i think it the vodka and coke that wipe my mind all the time think im gonna have to find a new drink'
 ... 'is eager for monday afternoon'
 "hope everyon and their mother had a great day! can't wait to hear what the guy have in store tomorrow!"
 'i love wake up to folgers. too bad my voic wa deeper than his.']


In [ ]:
print(X_test)

["@mmangen - m do fine! i haven't had much time to chat on twitter. hubbi is back for the summer &amp; tend to domin my free time."
 'at ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb it wa onli a bay area thang dammit!' ...
 '@destini41 nevertheless hooray! for 4700 member and have a wonder and safe trip!'
 'not feel too well' '@supersandro !!!! thank you!']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


AttributeError: 'TfidfVectorizer' object has no attribute 'fit_transfor'